This notebook contains the data analysis for RQ2

In [1]:
#Configuration

#Importing libraries
import json
import functools
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import pandas as pd
import datetime
import os

#DL9002 pip3

path= "./ground_truth_smells/"
file_names = []
manual_verification_results = []
smells = ["DL3019", 
          "DL3032", 
          "DL3036", 
          "DL3040", 
          #"DL3042", 
          "DL3060", 
          "DL9008", 
          "DL9009", 
          "DL9010", 
          "DL9011", 
          "DL9012",
          "DL3015",
          "DL9006",
          "DL9007",
          "DL3014",
          "DL3030",
          #"DL3034",
          "DL3038",
          "DL9003",
          "DL9004",
          "DL9005",
          "DL3008",
          #"DL3013",
          "DL3016",
          "DL3018",
          "DL3033",
          "DL3037",
          "DL3041",
          "DL9001",
          "DL9002",
          "DL9000",
          "DL9020",
          "DL9013",
          "DL9014",
          "DL9015",
          "DL9016",
          "DL9017",
          "DL9018",
          "DL9019",
          "DL3010",
          "DL3059"]

tool_smells_precision = [0] * len(smells)
tool_smells_recall = [0] * len(smells)
tool_smells_f1 = [0] * len(smells)
hadolint_smells_precision = [0] * len(smells)
hadolint_smells_recall = [0] * len(smells)
hadolint_smells_f1 = [0] * len(smells)
tool_detections = [0] * len(smells)
hadolint_detections = [0] * len(smells)






In [2]:
#load in all file objects
for root, _, files in os.walk(path):
    print(len(files))
    for file in files:
        file_path = os.path.join(path, file)
        with open(file_path) as f:
            data = json.load(f)
            file_names.append(file)
            manual_verification_results.append(data)
            
            

441


In [3]:
#Calculate
def division(a, b):
    return a / b if b else 0

for idx, smell in enumerate(smells):
    tool_true_positives = 0
    tool_false_positives = 0
    tool_false_negatives = 0
    
    hadolint_true_positives = 0
    hadolint_false_positives = 0
    hadolint_false_negatives = 0

    
    for result in manual_verification_results:
        truth_instances = 0
        tool_instances = 0
        hadolint_instances = 0
        
        # Get instances detected by the different instances
        if smell in result["File"]:
            truth_instances = result["File"][smell]
        if smell in result["Tool"]:
            tool_instances = result["Tool"][smell]
        if smell in result["Hadolint"]: 
            hadolint_instances = result["Hadolint"][smell]
            
        # Handle the tool
        if truth_instances == tool_instances:
            tool_true_positives += tool_instances
        elif truth_instances < tool_instances:
            tool_true_positives += truth_instances
            tool_false_positives += (tool_instances - truth_instances)
        else:
            tool_true_positives += tool_instances
            tool_false_negatives += (truth_instances - tool_instances)
        
        # Handle Hadolint
        if truth_instances == hadolint_instances:
            hadolint_true_positives += hadolint_instances
        elif truth_instances < hadolint_instances:
            hadolint_true_positives += truth_instances
            hadolint_false_positives += (hadolint_instances - truth_instances)
        else:
            hadolint_true_positives += hadolint_instances
            hadolint_false_negatives += (truth_instances - hadolint_instances)


    tool_precision = division(tool_true_positives,(tool_true_positives + tool_false_positives))
    tool_recall = division(tool_true_positives, (tool_true_positives + tool_false_negatives))
    tool_f1 = 2 * division((tool_precision * tool_recall), (tool_precision + tool_recall))
    
    hadolint_precision = division(hadolint_true_positives,(hadolint_true_positives + hadolint_false_positives))
    hadolint_recall = division(hadolint_true_positives, (hadolint_true_positives + hadolint_false_negatives))
    hadolint_f1 = 2 * division((hadolint_precision * hadolint_recall),(hadolint_precision + hadolint_recall))
    
    tool_detections[idx] = tool_true_positives + tool_false_positives
    tool_smells_precision[idx] = tool_precision
    tool_smells_recall[idx] = tool_recall
    tool_smells_f1[idx] = tool_f1
    hadolint_detections[idx] = hadolint_true_positives + hadolint_false_positives
    hadolint_smells_precision[idx] = hadolint_precision
    hadolint_smells_recall[idx] = hadolint_recall
    hadolint_smells_f1[idx] = hadolint_f1

        
    
smell_d = {"smells": smells, 
           "Tool detections": tool_detections,
           "Tool precision": tool_smells_precision,
           "Tool recall": tool_smells_recall,
           "Tool F1": tool_smells_f1,
           "Hadolint detections": hadolint_detections,
           "Hadolint precision": hadolint_smells_precision,
           "Hadolint recall": hadolint_smells_recall,
           "Hadolint F1": hadolint_smells_f1
          }
df = pd.DataFrame(smell_d)
print(df)
df.to_csv('results.txt', index=None, sep=' ', mode='w')

    smells  Tool detections  Tool precision  Tool recall   Tool F1  \
0   DL3019               14        1.000000     1.000000  1.000000   
1   DL3032               31        1.000000     0.968750  0.984127   
2   DL3036               15        1.000000     0.937500  0.967742   
3   DL3040                7        0.857143     1.000000  0.923077   
4   DL3060                3        1.000000     1.000000  1.000000   
5   DL9008               30        1.000000     1.000000  1.000000   
6   DL9009               17        0.941176     1.000000  0.969697   
7   DL9010                3        1.000000     1.000000  1.000000   
8   DL9011              192        0.937500     1.000000  0.967742   
9   DL9012                3        1.000000     0.750000  0.857143   
10  DL3015              152        0.993421     0.993421  0.993421   
11  DL9006                4        1.000000     1.000000  1.000000   
12  DL9007               14        1.000000     1.000000  1.000000   
13  DL3014          